<a href="https://colab.research.google.com/github/DipankarJDutta/DipankarJDutta-Biomarker-discovery-from-B-T-cells-in-FASD-GDM-with-LSTM/blob/Feature-extraction-with-Shap/LSTM_B_T_Median_optimized%20model_SHAP%20feature%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Don't display "warnings"
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [17]:
#Installing SHAP (SHapley Additive exPlanations)
!pip install shap

In [0]:
#Import SHAP specific essentials
from __future__ import print_function
from keras.preprocessing import sequence
from keras.layers import Embedding


In [0]:
# Loading essentials
from numpy import loadtxt
from numpy import reshape
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [0]:
# Fixing random seed to 007 for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [0]:
# Load data
dataset = loadtxt('B-T-median.csv', delimiter = ',')

In [0]:
# Reshape 2D dataset into a 3D dataset with columns as features with one time-step
x = dataset.reshape(56, 1, 30)

In [0]:
#Specify binary output of good (1) and bad (0) learners, y, in x. Learner type differentiated by population median.
y = x [:, :, -1]

In [0]:
# Define LSTM One-to-One Model with 3 LSTM cells (determined via hyperparameter optimization). Sigmoid activation on the output layer for binary classification.
model = Sequential()
model.add(LSTM(3, input_shape=(1,30)))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
#Fit the model with a 80-20 split of dataset & shuffling sample order within an epoch. Batch size updated to 8 to be an even factor of sample size of 56.
model.fit(x, y, validation_split = 0.2, batch_size = 8, epochs = 1000, shuffle = True, verbose=0)

In [0]:
#Explain model with DeepExplainer from SHAP
import sklearn
import shap
explainer = shap.DeepExplainer(model, x)
shap_values = explainer.shap_values(y)